## 1. 라이브러리

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2

from tqdm import tqdm
from sklearn.cluster import KMeans
from PIL import Image

## 2. 메타데이터 불러오기

In [2]:
dpath = 'data/' # 데이터 경로 설정
data = pd.read_csv(dpath + 'metadata.csv')
data

,album_id,album_name,artist,album_release_date,album_total_tracks,emotion,playlist_name
0,0025Y5D7IOZVpEfQtpvHIi,Into the Surf,Foals,2019-09-25,3,relaxed,calm winter morning ☆☆
1,002hMu6uJDan1XKVp3pmnb,Becoming,Feather,2021-01-13,5,sad,Sad Beats
2,002mOl8os2F1mDYRz8qU2a,Антигерой,ELMAN,2019-06-06,1,sad,슬픔
3,004EYz2DQttcGvyTQGDmLp,Merry Christmas Baby / White Christmas,Otis Redding,1968,2,happy,Happy Holidays
4,005KGm28jIUE3hW019Itx6,Too Much Love,Mujo,2021-04-15,1,relaxed,"Chill Beats 💙 groove, relax"
...,...,...,...,...,...,...,...
63309,7zw73YMhMg7lnMrkUAzE8t,Just Another Memory,JustSidekicks,2020-07-10,1,happy,Happy Vibes ?��
63310,7zwLfr8xSz9rWPU5KNBHkQ,cherry,Oatmello,2019-05-24,1,relaxed,Relax Lofi 😴
63311,7zxLuEY7mKLKdyWjnCfP35,Colorblind (feat. Dayseeker),Adventure Club,2021-08-13,1,sad,Sad Beats
63312,7zxfQk44mX0eW4eWtdDlKI,Mind Right,TK N Cash,2014-10-14,1,angry,Pissed Off Workout


#### OPENCV로 이미지 컬러 평균 추출하기

https://inyl.github.io/programming/2017/07/31/opencv_image_color_cluster.html

In [4]:
def centroid_histogram(clt):
    # grab the number of different clusters and create a histogram
    # based on the number of pixels assigned to each cluster
    numLabels = np.arange(0, len(np.unique(clt.labels_)) + 1)
    (hist, _) = np.histogram(clt.labels_, bins=numLabels)

    # normalize the histogram, such that it sums to one
    hist = hist.astype("float")
    hist /= hist.sum()

    # return the histogram
    return hist

## Kmeans 기반으로 추출한 대표색 5개와 그 비중을 리스트로 저장

In [ ]:
SENTIMENT = 'angry'   # happy, sad, relaxed, angry 에 대해 모두 실행

In [3]:
image_list = []
image_array_list = []
na_id = []
album_ids = data[data['emotion'] == SENTIMENT]['album_id']

for album_id in album_ids:
    image_list.append(Image.open(dpath + "emotion_album_images/{}/{}.jpg".format(SENTIMENT, album_id)).resize((300,300)))
    image_array_list.append(np.array(image_list[-1]))

In [5]:
# 모든 이미지에 대해서 계산
k = 5
center = []
hist = []

for image in tqdm(image_array_list):
    
    image = image.reshape((-1, 3))
    clt = KMeans(n_clusters = k)
    clt.fit(image)
    
    center.append(clt.cluster_centers_)
    
    h = centroid_histogram(clt)
    hist.append(h)

  2%|█▏                                                                          | 202/13206 [05:24<6:20:42,  1.76s/it]C:\Users\tnfls\AppData\Local\Temp/ipykernel_23936/3965629115.py:11: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  clt.fit(image)
  4%|██▋                                                                         | 470/13206 [12:40<5:06:29,  1.44s/it]C:\Users\tnfls\AppData\Local\Temp/ipykernel_23936/3965629115.py:11: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  clt.fit(image)
  4%|███                                                                         | 525/13206 [14:16<5:51:03,  1.66s/it]C:\Users\tnfls\AppData\Local\Temp/ipykernel_23936/3965629115.py:11: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (5). Possibly due to duplicate points in X.
  clt.fit(image)
  9%|█████

In [6]:
import pickle

with open('angry_center.pickle', 'wb') as f:
    pickle.dump(center, f)

with open('angry_hist.pickle', 'wb') as f:
    pickle.dump(hist, f)

In [ ]:
# exception이 발생한 인덱스 저장
except_list = [202, 470, 525, 1138, 2784, 2994, 4283, 4672, 5005, 5262, 
               5322, 7244, 7747, 7760, 7780, 8834, 9880, 10414, 11883, 11967]  # 이미지가 그냥 흰색이거나 검은색이어서 5개의 대표색이 추출되지 않은 이미지 인덱스